In [1]:
import numpy as np
import sympy

In [2]:
class SimplexSolver:
    def __init__(self, A, b, c):
        self.A = A  # size m x n
        self.b = b  # size m
        self.c = c  # size n
        self.n = c.shape[0]
        self.m = A.shape[0]
        self.tableau = None
        self.basic_indices = None
        self.nonbasic_indices = None
        self.type = None

    def solve(self):
        self.build_tableau()
        first_row_min = np.min(self.tableau[0, 1:])
        first_col_min = np.min(self.tableau[1:, 0])
        if first_row_min < 0 and first_col_min < 0:
            print("Initial tableau is not feasible or dual feasible")
            return
        elif first_row_min >= 0 and first_col_min >= 0:
            print("Initial tableau is optimal")
            self.print_tableau()
            self.print_solution()
            return
        elif first_row_min >= 0:
            print("Initial tableau is dual feasible")
            self.print_tableau()
            self.type = "dual"
        else:
            print("Initial tableau is feasible")
            self.print_tableau()
            self.type = "prime"

        while self.check_tableau():
            i, j = self.select_pivot()
            if i == 0 or j == 0:
                return
            self.transform_tableau(i - 1, j - 1)
            self.print_tableau()
        self.print_solution()

    def build_tableau(self):
        # find indices of independent columns in A
        _, ids = sympy.Matrix(self.A).rref()

        # basic and nonbasic indices of A
        basic_ids = list(ids)
        nonbasic_ids = [i for i in range(self.n) if i not in basic_ids]

        # vector c_B and matrix A_B
        c_basic = self.c[basic_ids]
        A_basic = self.A[:, basic_ids]

        # vector c_N and matrix A_N
        c_nonbasic = self.c[nonbasic_ids]
        A_nonbasic = self.A[:, nonbasic_ids]

        # Create matrices \tilde{A}_B, \tilde{A}_N and \tilde{b}
        unit_vector = np.append(np.ones(1), np.zeros(self.m))
        A_new_basic = np.column_stack((unit_vector, np.vstack((c_basic, A_basic))))
        A_new_nonbasic = np.vstack((c_nonbasic, A_nonbasic))
        b_new = np.append(np.zeros(1), self.b)

        # Inverse matrix of \tilde{A}_B
        A_new_inv = np.linalg.inv(A_new_basic)

        # Create simplex tableau
        first_column = np.matmul(A_new_inv, b_new)
        other_columns = np.matmul(A_new_inv, A_new_nonbasic)
        self.tableau = np.column_stack((first_column, other_columns))

        # Save indices
        self.basic_indices = [i + 1 for i in basic_ids]
        self.nonbasic_indices = [i + 1 for i in nonbasic_ids]

    def select_pivot(self):
        if self.type == "prime":
            negative_nonbasic = np.argwhere(self.tableau[0, 1:] < 0)
            j = negative_nonbasic[0][0] + 1
            if np.argwhere(self.tableau[1:, j] <= 0).shape[0] == self.m:
                print("Problem is unbounded")
                return 0, j
            valid_idx = np.where(self.tableau[1:, j] > 0)[0] + 1
            i = valid_idx[np.divide(self.tableau[valid_idx, 0], self.tableau[valid_idx, j]).argmin()]
            return i, j
        elif self.type == "dual":
            negative_basic = np.argwhere(self.tableau[1:, 0] < 0)
            i = negative_basic[0][0] + 1
            if np.argwhere(self.tableau[i, 1:] >= 0).shape[0] == self.n - self.m:
                print("Problem is unbounded")
                return i, 0
            valid_idx = np.where(self.tableau[i, 1:] < 0)[0] + 1
            j = valid_idx[np.negative(np.divide(self.tableau[0, valid_idx], self.tableau[i, valid_idx])).argmin()]
            return i, j

    def transform_tableau(self, i, j):
        self.basic_indices[i], self.nonbasic_indices[j] = self.nonbasic_indices[j], self.basic_indices[i]
        i, j = i + 1, j + 1
        B = [k for k in range(self.m + 1) if k != i]
        N = [k for k in range(self.n - self.m + 1) if k != j]
        for p in B:
            for q in N:
                self.tableau[p][q] -= self.tableau[p][j] * self.tableau[i][q] / self.tableau[i][j]
        for p in B:
            self.tableau[p][j] = (-1) * self.tableau[p][j] / self.tableau[i][j]
        for q in N:
            self.tableau[i][q] = self.tableau[i][q] / self.tableau[i][j]
        self.tableau[i][j] = 1. / self.tableau[i][j]

    def print_solution(self):
        row_ids = {self.basic_indices[i]: i + 1 for i in range(self.m)}
        x = [self.tableau[row_ids[i + 1]][0] if i + 1 in self.basic_indices else 0 for i in range(self.n)]
        print("-"*20)
        print("Solution:")
        print("A:\n", self.A, sep="")
        print("b:\n", self.b)
        print("c:\n", self.c)
        print("Minimum value is {0} with x = {1}".format(-self.tableau[0][0], x))

    def print_tableau(self):
        print("-"*20)
        print(self.tableau)
        print("B = %s" % self.basic_indices)
        print("N = %s" % self.nonbasic_indices)

    def check_tableau(self):
        if self.type == "prime":
            return np.min(self.tableau[0, 1:]) < 0
        elif self.type == "dual":
            return np.min(self.tableau[1:, 0]) < 0

In [3]:
# Test 1
A = np.array([[-1, 0, 1], [0, 1, 1]])
b = np.array([1, 2])
c = np.array([0, 0, 1])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is dual feasible
--------------------
[[ 0.  1.]
 [-1. -1.]
 [ 2.  1.]]
B = [1, 2]
N = [3]
--------------------
[[-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]
B = [3, 2]
N = [1]
--------------------
Solution:
A:
[[-1  0  1]
 [ 0  1  1]]
b:
 [1 2]
c:
 [0 0 1]
Minimum value is 1.0 with x = [0, 1.0, 1.0]


In [4]:
# Test 2
A = np.array([[1, -2, 1, 0], [2, 1, 0, 1]])
b = np.array([1, 6])
c = np.array([-4, 3, 0, 0])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is optimal
--------------------
[[ 8.   2.   1. ]
 [ 2.6  0.2  0.4]
 [ 0.8 -0.4  0.2]]
B = [1, 2]
N = [3, 4]
--------------------
Solution:
A:
[[ 1 -2  1  0]
 [ 2  1  0  1]]
b:
 [1 6]
c:
 [-4  3  0  0]
Minimum value is -8.0 with x = [2.6, 0.8, 0, 0]


In [5]:
# Test 3
A = np.array([[1, -2, 1, 0, 0], [2, 1, 0, 1, 0], [1, 0, 0, 0, 1]])
b = np.array([1, 6, 2])
c = np.array([-4, 3, 0, 0, 0])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is feasible
--------------------
[[ 2. -3. 10.]
 [ 2.  0.  1.]
 [ 2.  1. -2.]
 [ 3.  2. -5.]]
B = [1, 2, 3]
N = [4, 5]
--------------------
[[ 6.5  1.5  2.5]
 [ 2.  -0.   1. ]
 [ 0.5 -0.5  0.5]
 [ 1.5  0.5 -2.5]]
B = [1, 2, 4]
N = [3, 5]
--------------------
Solution:
A:
[[ 1 -2  1  0  0]
 [ 2  1  0  1  0]
 [ 1  0  0  0  1]]
b:
 [1 6 2]
c:
 [-4  3  0  0  0]
Minimum value is -6.5 with x = [2.0, 0.5, 0, 1.5, 0]


In [6]:
# Test 4
A = np.array([[2, 3, 1, 1, 0, 0], [4, 1, 2, 0, 1, 0], [3, 4, 2, 0, 0, 1]])
b = np.array([5, 11, 8])
c = np.array([-5, -4, -3, 0, 0, 0])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is not feasible or dual feasible


In [7]:
# Test 5
A = np.array([[1, 3, 1, 1, 0, 0, 0], [-1, 0, 3, 0, 1, 0, 0], [2, -1, 2, 0, 0, 1, 0], [2, 3, -1, 0, 0, 0, 1]])
b = np.array([3, 2, 4, 2])
c = np.array([-5, -5, -3, 0, 0, 0, 0])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is optimal
--------------------
[[10.          1.          1.          2.        ]
 [ 1.10344828 -0.17241379  0.31034483  0.10344828]
 [ 0.27586207  0.20689655 -0.17241379  0.27586207]
 [ 1.03448276  0.27586207  0.10344828  0.03448276]
 [ 0.03448276 -0.72413793  0.10344828 -0.96551724]]
B = [1, 2, 3, 4]
N = [5, 6, 7]
--------------------
Solution:
A:
[[ 1  3  1  1  0  0  0]
 [-1  0  3  0  1  0  0]
 [ 2 -1  2  0  0  1  0]
 [ 2  3 -1  0  0  0  1]]
b:
 [3 2 4 2]
c:
 [-5 -5 -3  0  0  0  0]
Minimum value is -10.0 with x = [1.103448275862069, 0.2758620689655172, 1.0344827586206895, 0.0344827586206895, 0, 0, 0]


In [8]:
# Test 6
A = np.array([[-2, 1, 1]])
b = np.array([1])
c = np.array([-1, 1, 0])
solver = SimplexSolver(A, b, c)
solver.solve()

Initial tableau is not feasible or dual feasible


In [9]:
class TwoPhasesSimplexSolver:
    def __init__(self, A, b, c):
        self.A = A  # size m x n
        self.b = b  # size m
        self.c = c  # size n
        self.n = c.shape[0]
        self.m = A.shape[0]
        self.tableau = None
        self.basic_indices = None
        self.nonbasic_indices = None
        self.type = None

    def solve(self):
        print("Start phase 1")
        self.build_tableau()
        self.print_tableau()

        while not set(range(self.n + 1, self.n + self.m + 1)).issubset(set(self.nonbasic_indices)):
            i, j = self.select_pivot()
            if j == 0:
                return
            self.transform_tableau(i - 1, j - 1)
            self.print_tableau()
        cols_to_keep = [0]
        new_nonbasic_indices = []
        for i in range(self.n):
            if self.nonbasic_indices[i] <= self.n:
                cols_to_keep.append(i + 1)
                new_nonbasic_indices.append(self.nonbasic_indices[i])
        rows_to_keep = [self.m + 1] + list(range(1, self.m + 1))
        self.tableau = self.tableau[rows_to_keep, :][:, cols_to_keep]
        self.nonbasic_indices = new_nonbasic_indices

        self.print_tableau()
        print("This is the tableau after phase 1")
        print("*" * 20)
        print("Start phase 2")
        while np.min(self.tableau[0, 1:]) < 0:
            i, j = self.select_pivot()
            if i == 0 or j == 0:
                return
            self.transform_tableau(i - 1, j - 1)
            self.print_tableau()

        self.print_solution()

    def build_tableau(self):
        """
            Build the table for an auxiliary problem
                    -1^T b     -1^T A
            T =        b          A
                       0         c^T
        """
        self.basic_indices = list(range(self.n + 1, self.n + self.m + 1))
        self.nonbasic_indices = list(range(1, self.n + 1))
        first_column = np.hstack((np.negative(np.dot(np.ones(self.m), self.b)), self.b, np.array([0])))
        other_columns = np.vstack((np.negative(np.matmul(np.ones(self.m), self.A)), self.A, self.c))
        self.tableau = np.column_stack((first_column, other_columns))

    def select_pivot(self):
        negative_nonbasic = np.argwhere(self.tableau[0, 1:] < 0)
        j = negative_nonbasic[0][0] + 1
        if np.argwhere(self.tableau[1:self.m+1, j] <= 0).shape[0] == self.m:
            print("Problem is unbounded")
            return 0, j
        valid_idx = np.where(self.tableau[1:self.m+1, j] > 0)[0] + 1
        i = valid_idx[np.divide(self.tableau[valid_idx, 0], self.tableau[valid_idx, j]).argmin()]
        return i, j

    def transform_tableau(self, i, j):
        self.basic_indices[i], self.nonbasic_indices[j] = self.nonbasic_indices[j], self.basic_indices[i]
        i, j = i + 1, j + 1
        B = [k for k in range(self.tableau.shape[0]) if k != i]
        N = [k for k in range(self.tableau.shape[1]) if k != j]
        for p in B:
            for q in N:
                self.tableau[p][q] -= self.tableau[p][j] * self.tableau[i][q] / self.tableau[i][j]
        for p in B:
            self.tableau[p][j] = (-1) * self.tableau[p][j] / self.tableau[i][j]
        for q in N:
            self.tableau[i][q] = self.tableau[i][q] / self.tableau[i][j]
        self.tableau[i][j] = 1. / self.tableau[i][j]

    def print_solution(self):
        row_ids = {self.basic_indices[i]: i + 1 for i in range(self.m)}
        x = [self.tableau[row_ids[i + 1]][0] if i + 1 in self.basic_indices else 0 for i in range(self.n)]
        print("-"*20)
        print("Solution:")
        print("A:\n", self.A, sep="")
        print("b:\n", self.b)
        print("c:\n", self.c)
        print("Minimum value is {0} with x = {1}".format(-self.tableau[0][0], x))

    def print_tableau(self):
        print("-"*20)
        print(self.tableau)
        print("B = %s" % self.basic_indices)
        print("N = %s" % self.nonbasic_indices)

In [10]:
# Test 1
A = np.array([[-1, 0, 1], [0, 1, 1]])
b = np.array([1, 2])
c = np.array([0, 0, 1])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-3.  1. -1. -2.]
 [ 1. -1.  0.  1.]
 [ 2.  0.  1.  1.]
 [ 0.  0.  0.  1.]]
B = [4, 5]
N = [1, 2, 3]
--------------------
[[-1.  1.  1. -1.]
 [ 1. -1. -0.  1.]
 [ 2.  0.  1.  1.]
 [ 0.  0. -0.  1.]]
B = [4, 2]
N = [1, 5, 3]
--------------------
[[ 0.  0.  1.  1.]
 [ 1. -1. -0.  1.]
 [ 1.  1.  1. -1.]
 [-1.  1.  0. -1.]]
B = [3, 2]
N = [1, 5, 4]
--------------------
[[-1.  1.]
 [ 1. -1.]
 [ 1.  1.]]
B = [3, 2]
N = [1]
This is the tableau after phase 1
********************
Start phase 2
--------------------
Solution:
A:
[[-1  0  1]
 [ 0  1  1]]
b:
 [1 2]
c:
 [0 0 1]
Minimum value is 1.0 with x = [0, 1.0, 1.0]


In [11]:
# Test 2
A = np.array([[1, -2, 1, 0], [2, 1, 0, 1]])
b = np.array([1, 6])
c = np.array([-4, 3, 0, 0])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-7. -3.  1. -1. -1.]
 [ 1.  1. -2.  1.  0.]
 [ 6.  2.  1.  0.  1.]
 [ 0. -4.  3.  0.  0.]]
B = [5, 6]
N = [1, 2, 3, 4]
--------------------
[[-4.  3. -5.  2. -1.]
 [ 1.  1. -2.  1.  0.]
 [ 4. -2.  5. -2.  1.]
 [ 4.  4. -5.  4.  0.]]
B = [1, 6]
N = [5, 2, 3, 4]
--------------------
[[ 0.   1.   1.   0.   0. ]
 [ 2.6  0.2  0.4  0.2  0.4]
 [ 0.8 -0.4  0.2 -0.4  0.2]
 [ 8.   2.   1.   2.   1. ]]
B = [1, 2]
N = [5, 6, 3, 4]
--------------------
[[ 8.   2.   1. ]
 [ 2.6  0.2  0.4]
 [ 0.8 -0.4  0.2]]
B = [1, 2]
N = [3, 4]
This is the tableau after phase 1
********************
Start phase 2
--------------------
Solution:
A:
[[ 1 -2  1  0]
 [ 2  1  0  1]]
b:
 [1 6]
c:
 [-4  3  0  0]
Minimum value is -8.0 with x = [2.6, 0.8, 0, 0]


In [12]:
# Test 3
A = np.array([[1, -2, 1, 0, 0], [2, 1, 0, 1, 0], [1, 0, 0, 0, 1]])
b = np.array([1, 6, 2])
c = np.array([-4, 3, 0, 0, 0])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-9. -4.  1. -1. -1. -1.]
 [ 1.  1. -2.  1.  0.  0.]
 [ 6.  2.  1.  0.  1.  0.]
 [ 2.  1.  0.  0.  0.  1.]
 [ 0. -4.  3.  0.  0.  0.]]
B = [6, 7, 8]
N = [1, 2, 3, 4, 5]
--------------------
[[-5.  4. -7.  3. -1. -1.]
 [ 1.  1. -2.  1.  0.  0.]
 [ 4. -2.  5. -2.  1.  0.]
 [ 1. -1.  2. -1.  0.  1.]
 [ 4.  4. -5.  4.  0.  0.]]
B = [1, 7, 8]
N = [6, 2, 3, 4, 5]
--------------------
[[-1.5  0.5  3.5 -0.5 -1.   2.5]
 [ 2.   0.   1.   0.   0.   1. ]
 [ 1.5  0.5 -2.5  0.5  1.  -2.5]
 [ 0.5 -0.5  0.5 -0.5  0.   0.5]
 [ 6.5  1.5  2.5  1.5  0.   2.5]]
B = [1, 7, 2]
N = [6, 8, 3, 4, 5]
--------------------
[[ 0.  1.  1.  1.  0.  0.]
 [ 2.  0.  1. -0.  0.  1.]
 [ 3.  1. -5.  2.  2. -5.]
 [ 2.  0. -2.  1.  1. -2.]
 [ 2.  0. 10. -3. -3. 10.]]
B = [1, 3, 2]
N = [6, 8, 7, 4, 5]
--------------------
[[ 2. -3. 10.]
 [ 2.  0.  1.]
 [ 3.  2. -5.]
 [ 2.  1. -2.]]
B = [1, 3, 2]
N = [4, 5]
This is the tableau after phase 1
********************
Start phase 2
----------------

In [13]:
# Test 4
A = np.array([[2, 3, 1, 1, 0, 0], [4, 1, 2, 0, 1, 0], [3, 4, 2, 0, 0, 1]])
b = np.array([5, 11, 8])
c = np.array([-5, -4, -3, 0, 0, 0])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-24.  -9.  -8.  -5.  -1.  -1.  -1.]
 [  5.   2.   3.   1.   1.   0.   0.]
 [ 11.   4.   1.   2.   0.   1.   0.]
 [  8.   3.   4.   2.   0.   0.   1.]
 [  0.  -5.  -4.  -3.   0.   0.   0.]]
B = [7, 8, 9]
N = [1, 2, 3, 4, 5, 6]
--------------------
[[-1.5  4.5  5.5 -0.5  3.5 -1.  -1. ]
 [ 2.5  0.5  1.5  0.5  0.5  0.   0. ]
 [ 1.  -2.  -5.   0.  -2.   1.   0. ]
 [ 0.5 -1.5 -0.5  0.5 -1.5  0.   1. ]
 [12.5  2.5  3.5 -0.5  2.5  0.   0. ]]
B = [1, 8, 9]
N = [7, 2, 3, 4, 5, 6]
--------------------
[[-1.  3.  5.  1.  2. -1.  0.]
 [ 2.  2.  2. -1.  2.  0. -1.]
 [ 1. -2. -5. -0. -2.  1.  0.]
 [ 1. -3. -1.  2. -3.  0.  2.]
 [13.  1.  3.  1.  1.  0.  1.]]
B = [1, 8, 3]
N = [7, 2, 9, 4, 5, 6]
--------------------
[[ 0.  1.  0.  1.  0.  1.  0.]
 [ 2.  2.  2. -1.  2. -0. -1.]
 [ 1. -2. -5. -0. -2.  1.  0.]
 [ 1. -3. -1.  2. -3. -0.  2.]
 [13.  1.  3.  1.  1. -0.  1.]]
B = [1, 5, 3]
N = [7, 2, 9, 4, 8, 6]
--------------------
[[13.  3.  1.  1.]
 [ 2.  2.  2. -1.]
 

In [14]:
# Test 5
A = np.array([[1, 3, 1, 1, 0, 0, 0], [-1, 0, 3, 0, 1, 0, 0], [2, -1, 2, 0, 0, 1, 0], [2, 3, -1, 0, 0, 0, 1]])
b = np.array([3, 2, 4, 2])
c = np.array([-5, -5, -3, 0, 0, 0, 0])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-11.  -4.  -5.  -5.  -1.  -1.  -1.  -1.]
 [  3.   1.   3.   1.   1.   0.   0.   0.]
 [  2.  -1.   0.   3.   0.   1.   0.   0.]
 [  4.   2.  -1.   2.   0.   0.   1.   0.]
 [  2.   2.   3.  -1.   0.   0.   0.   1.]
 [  0.  -5.  -5.  -3.   0.   0.   0.   0.]]
B = [8, 9, 10, 11]
N = [1, 2, 3, 4, 5, 6, 7]
--------------------
[[-7.   2.   1.  -7.  -1.  -1.  -1.   1. ]
 [ 2.  -0.5  1.5  1.5  1.   0.   0.  -0.5]
 [ 3.   0.5  1.5  2.5  0.   1.   0.   0.5]
 [ 2.  -1.  -4.   3.   0.   0.   1.  -1. ]
 [ 1.   0.5  1.5 -0.5  0.   0.   0.   0.5]
 [ 5.   2.5  2.5 -5.5  0.   0.   0.   2.5]]
B = [8, 9, 10, 1]
N = [11, 2, 3, 4, 5, 6, 7]
--------------------
[[-2.33333333 -0.33333333 -8.33333333  2.33333333 -1.         -1.
   1.33333333 -1.33333333]
 [ 1.          0.          3.5        -0.5         1.          0.
  -0.5         0.        ]
 [ 1.33333333  1.33333333  4.83333333 -0.83333333  0.          1.
  -0.83333333  1.33333333]
 [ 0.66666667 -0.33333333 -1.3333333

In [15]:
# Test 6
A = np.array([[-2, 1, 1]])
b = np.array([1])
c = np.array([-1, 1, 0])
solver = TwoPhasesSimplexSolver(A, b, c)
solver.solve()

Start phase 1
--------------------
[[-1.  2. -1. -1.]
 [ 1. -2.  1.  1.]
 [ 0. -1.  1.  0.]]
B = [4]
N = [1, 2, 3]
--------------------
[[ 0.  0.  1.  0.]
 [ 1. -2.  1.  1.]
 [-1.  1. -1. -1.]]
B = [2]
N = [1, 4, 3]
--------------------
[[-1.  1. -1.]
 [ 1. -2.  1.]]
B = [2]
N = [1, 3]
This is the tableau after phase 1
********************
Start phase 2
--------------------
[[ 0. -1.  1.]
 [ 1. -2.  1.]]
B = [3]
N = [1, 2]
Problem is unbounded
